<a href="https://colab.research.google.com/github/ariyanzri/tumor_segmentation/blob/master/Data_Split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf /content/sample_data

In [4]:
!gdown --id 1CyK9bRu6jwriYu5fVrP_gTnjlLI3Gkvz

Downloading...
From: https://drive.google.com/uc?id=1CyK9bRu6jwriYu5fVrP_gTnjlLI3Gkvz
To: /content/patch_data.tar.gz
14.4GB [02:07, 113MB/s] 


In [5]:
print('Download complete, Now extracting...')
!tar xvf /content/patch_data.tar.gz

Streaming output truncated to the last 5000 lines.
complete_dataset_no_duplicate/normal_040/normal/normal_040-patch_869.jpg
complete_dataset_no_duplicate/normal_040/normal/normal_040-patch_987.jpg
complete_dataset_no_duplicate/normal_040/normal/normal_040-patch_1629.jpg
complete_dataset_no_duplicate/normal_040/normal/normal_040-patch_867.jpg
complete_dataset_no_duplicate/normal_040/normal/normal_040-patch_378.jpg
complete_dataset_no_duplicate/normal_040/normal/normal_040-patch_1203.jpg
complete_dataset_no_duplicate/normal_040/normal/normal_040-patch_744.jpg
complete_dataset_no_duplicate/normal_040/normal/normal_040-patch_1265.jpg
complete_dataset_no_duplicate/normal_040/normal/normal_040-patch_657.jpg
complete_dataset_no_duplicate/normal_040/normal/normal_040-patch_558.jpg
complete_dataset_no_duplicate/normal_040/normal/normal_040-patch_1684.jpg
complete_dataset_no_duplicate/normal_040/normal/normal_040-patch_1845.jpg
complete_dataset_no_duplicate/normal_040/normal/normal_040-patch_103

In [6]:
print('Extract Complete')

Extract Complete


In [7]:
import os
import numpy as np

normals = []
abnormals = []
total = os.listdir('/content/complete_dataset_no_duplicate')
for filename in total:
    if 'normal' in filename:
        normals.append(filename)
    else:
        abnormals.append(filename)

np.random.shuffle(normals)
np.random.shuffle(abnormals)

n_normal = len(normals)
n_abnormal = len(abnormals)

print('Total normal patients: ', n_normal)
print('Total abnormal patients: ', n_abnormal)

n = n_normal + n_abnormal
print('Total patients: ', n)

Total normal patients:  159
Total abnormal patients:  111
Total patients:  270


In [8]:
ratio = {'train':0.6, 'test':0.2, 'dev':0.2}

data = {}

last_index_n = 0
last_index_ab = 0

train_n_abnormal =  int(np.ceil(ratio['train'] * n_abnormal))
train_n_normal =  int(np.ceil(ratio['train'] * n_normal))

train_set = normals[last_index_n : train_n_normal] + abnormals[last_index_ab : train_n_abnormal]
np.random.shuffle(train_set)
data['train'] = train_set
last_index_n += train_n_normal
last_index_ab += train_n_abnormal

test_n_abnormal =  int(np.ceil(ratio['test'] * n_abnormal))
test_n_normal =  int(np.ceil(ratio['test'] * n_normal))

test_set = normals[last_index_n : last_index_n + test_n_normal] + abnormals[last_index_ab  : last_index_ab + test_n_abnormal]
np.random.shuffle(test_set)
data['test'] = test_set
last_index_n += test_n_normal
last_index_ab += test_n_abnormal

dev_set = normals[last_index_n:] + abnormals[last_index_ab:]
np.random.shuffle(dev_set)
data['dev'] = dev_set

print('Total: ', n)
print('Train: ', len(data['train']))
print('Test:', len(data['test']))
print('Dev:', len(data['dev']))

### Integrity check
sum = []
for _, value in data.items():
    sum += (value)

intersection_1 = len(list(set(data['train']) & set(data['test'])))
intersection_2 = len(list(set(data['train']) & set(data['dev'])))
intersection_3 = len(list(set(data['test']) & set(data['dev'])))

duplicate_train = len(list(set(data['train']))) == len(data['train'])
duplicate_test = len(list(set(data['test']))) == len(data['test'])
duplicate_dev = len(list(set(data['dev']))) == len(data['dev'])

if set(sum) == set(total) and intersection_1 == 0 and intersection_2 == 0 and intersection_3 == 0 and duplicate_test and duplicate_train and duplicate_dev:
    print('\nIntegrity check: OK')
    print('Train + Test + Dev == Total : ', n == (len(data['train']) + len(data['test']) + len(data['dev'])))
else:
    print('\nIntegrity check: Fail')

Total:  270
Train:  163
Test: 55
Dev: 52

Integrity check: OK
Train + Test + Dev == Total :  True


In [9]:
from shutil import copy2

target_dir = '/content/Data/'
parent_dir = '/content/complete_dataset_no_duplicate/'

file_counter = 0
data_y = {}

target_path = os.path.join(target_dir, 'output')
os.makedirs(target_path, exist_ok = True)

for key, value in data.items():
    data_y[key] = []
    target_path = os.path.join(target_dir, key)
    os.makedirs(target_path, exist_ok = True)
    for folder in value:
        for subfolder in os.listdir(parent_dir + folder + '/'):
            for filename in os.listdir(parent_dir + folder + '/' + subfolder + '/'):
                if 'normal' in filename:
                    data_y[key].append(1)
                elif 'tumor' in filename:
                    data_y[key].append(0)
                src = os.path.join(parent_dir, folder, subfolder, filename)
                dst = os.path.join(target_path, str(file_counter) +'.jpg')
                copy2(src, dst)
                file_counter += 1
    print('Total', key, 'examples: ', file_counter)
    file_counter = 0

    out_path = os.path.join(target_dir, 'output', key + '_y.npy')
    np.save(out_path, np.array(data_y[key]))

print('Copy Complete')

Total train examples:  310094
Total test examples:  103800
Total dev examples:  96883
Copy Complete


In [10]:
import subprocess
for folder in os.listdir(target_dir):
    path = os.path.join(target_dir, folder)
    zip_name = folder + '_patch_data'
    subprocess.call(['zip', '-r', zip_name, path])
    print(zip_name + '.zip created.')

dev_patch_data.zip created.
train_patch_data.zip created.
test_patch_data.zip created.
output_patch_data.zip created.


In [11]:
!unzip /content/output_patch_data.zip

Archive:  /content/output_patch_data.zip
   creating: content/Data/output/
  inflating: content/Data/output/train_y.npy  
  inflating: content/Data/output/dev_y.npy  
  inflating: content/Data/output/test_y.npy  


In [14]:
!cp /content/output_patch_data.zip '/content/drive/My Drive/IVILAB Data/'

In [15]:
!cp /content/test_patch_data.zip '/content/drive/My Drive/IVILAB Data/'

In [16]:
!cp /content/dev_patch_data.zip '/content/drive/My Drive/IVILAB Data/'

In [17]:
!cp /content/train_patch_data.zip '/content/drive/My Drive/IVILAB Data/'